In [ ]:
# faceMask and faceMask2 hDDM face ratings analysis script: mask analyses of false negatives (failing to ID correct emotion)
# 22/03/21

# note: to be run within python 3.5 environment named "hddm" (from console: 'source activate hddm')
# confirm jupyter notebook is launching from "hddm" environment (see top-right: 'conda env:hddm')

# set up
import numpy as np
print(np.__version__) # should be 1.11.3

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from patsy import dmatrix

import hddm
print(hddm.__version__) # should be 0.6.0

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data1 = hddm.load_csv('faceMask_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data1 = hddm.utils.flip_errors(data1)

# check dataset
data1.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data1.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (fearful) and facial expression (fearful)
ff1_data = data1.loc[(data1['emotionRating'] == 'fearful') & (data1['expression'] == 'fearful')]
ff1_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
ff1_data = ff1_data[(ff1_data.subj_idx != '7chb1ucy8l7g29z') & (ff1_data.subj_idx != 'ygpcur698h4dcw5')]

In [ ]:
# define model
m_ff1 = hddm.HDDMRegressor(ff1_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_ff1.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_ff1.print_stats()
m_ff1.plot_posteriors()

In [ ]:
# save model stats
ff1_stats = m_ff1.gen_stats()
print(ff1_stats)
ff1_stats.to_csv('ff1_5000/faceMask_hddm_drift_maskFalseNegatives_ff1_5000.csv', index=True)

# save model posterior plots
m_ff1.plot_posteriors(path='ff1_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
ff1_v_none, ff1_v_lower, ff1_v_upper = m_ff1.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([ff1_v_none, ff1_v_lower, ff1_v_upper])
plt.savefig('ff1_5000/faceMask_hddm_drift_maskFalsePositives_ff1_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(ff1_v_none > 0) = ", (ff1_v_none.trace() > 0).mean())
print("P(ff1_v_lower < 0) = ", (ff1_v_lower.trace() < 0).mean())
print("P(ff1_v_upper > 0) = ", (ff1_v_upper.trace() > 0).mean())
# estimate probability that lower and upper mask coefficient posteriors differ
print("P(ff1_v_lower < ff1_v_upper) = ", (ff1_v_lower.trace() < ff1_v_upper.trace()).mean())

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data2 = hddm.load_csv('faceMask2_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data2 = hddm.utils.flip_errors(data2)

# check dataset
data2.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data2.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (fearful) and facial expression (angry)
ff2_data = data2.loc[(data2['emotionRating'] == 'fearful') & (data2['expression'] == 'fearful')]
ff2_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
ff2_data = ff2_data[(ff2_data.subj_idx != 'me8doxrmo9vj9dx') & (ff2_data.subj_idx != 'n4v0blzwqwgrcpn') & (ff2_data.subj_idx != 'xs5439nm2v85thb')]

In [ ]:
# define model
m_ff2 = hddm.HDDMRegressor(ff2_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_ff2.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_ff2.print_stats()
m_ff2.plot_posteriors()

In [ ]:
# save model stats
ff2_stats = m_ff2.gen_stats()
print(ff2_stats)
ff2_stats.to_csv('ff2_5000/faceMask2_hddm_drift_maskFalseNegatives_ff2_5000.csv', index=True)

# save model posterior plots
m_ff2.plot_posteriors(path='ff2_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
ff2_v_none, ff2_v_lower, ff2_v_upper = m_ff2.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([ff2_v_none, ff2_v_lower, ff2_v_upper])
plt.savefig('ff2_5000/faceMask2_hddm_drift_maskFalsePositives_ff2_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(ff2_v_none < 0) = ", (ff2_v_none.trace() < 0).mean())
print("P(ff2_v_lower > 0) = ", (ff2_v_lower.trace() > 0).mean())
print("P(ff2_v_upper > 0) = ", (ff2_v_upper.trace() > 0).mean())
# estimate probability that lower and upper mask coefficient posteriors differ
print("P(ff2_v_lower < ff2_v_upper) = ", (ff2_v_lower.trace() < ff2_v_upper.trace()).mean())

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from each other (faceMask 1 vs. faceMask2)
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(ff1_v_none > ff2_v_none) = ", ((ff1_v_none.trace() > ff2_v_none.trace()).mean()))
print("P(ff1_v_lower < ff2_v_lower) = ", ((ff1_v_lower.trace() < ff2_v_lower.trace()).mean()))
print("P(ff1_v_upper < ff2_v_upper) = ", ((ff1_v_upper.trace() < ff2_v_upper.trace()).mean()))


In [ ]:
## adding together the relative values:
ff1_v_none_plus_lower = ff1_v_none.trace() + ff1_v_lower.trace()
ff1_v_none_plus_upper = ff1_v_none.trace() + ff1_v_upper.trace()
ff2_v_none_plus_lower = ff2_v_none.trace() + ff2_v_lower.trace()
ff2_v_none_plus_upper = ff2_v_none.trace() + ff2_v_upper.trace()

In [ ]:
print("P(ff1_v_none_plus_lower > ff2_v_none_plus_lower) = ", (ff1_v_none_plus_lower > ff2_v_none_plus_lower).mean())
print("P(ff1_v_none_plus_upper > ff2_v_none_plus_upper) = ", (ff1_v_none_plus_upper > ff2_v_none_plus_upper).mean())